In [1]:
import pandas as pd

# 학교의 기본 정보를 가져온다

In [54]:
df1=pd.read_excel('학교개황(20220310기준).xls').loc[:,'학교명':'학교홈페이지']
df1=df1[['학교명']+list(df1.columns[2:5])+['주소','학교홈페이지']]
df1=df1[(df1['학제']=="대학교") |(df1['학제']=="전문대학")]
df1['학교명']=df1['학교명'].apply(lambda x : str(x).split('(')[0])
df1['주소']=df1['주소'].apply(lambda x : str(x).split(' ')[1])
df1=df1[(df1['설립구분']=="국립") |(df1['설립구분']=="사립")]
df1.rename(columns={'지역':'시','주소':'지역'},inplace=True)
data={}
for i in df1['시'].unique():
    data[i]=list(df1[df1['시']==i]['지역'].unique())
df1=df1.groupby('학교명').agg(
    {
        '학제' : 'first',
        '설립구분':'first',
        '시' : ", ".join,
        '지역' : ", ".join,
        '학교홈페이지' : 'first',
    }
).reset_index()
df1['시']=df1['시'].apply(lambda x : ", ".join(list(set(str(x).split(', ')))))
df1['학교홈페이지']=df1['학교홈페이지'].apply(lambda x : str(x).lower().split('/')[0])
df1

,학교명,학제,설립구분,시,지역,학교홈페이지
0,가야대학교,대학교,사립,"경북, 경남","김해시, 고령군",www.kaya.ac.kr
1,가천대학교,대학교,사립,경기,성남시,www.gachon.ac.kr
2,가톨릭관동대학교,대학교,사립,강원,강릉시,www.cku.ac.kr
3,가톨릭꽃동네대학교,대학교,사립,충북,청주시,www.kkot.ac.kr
4,가톨릭대학교,대학교,사립,"경기, 서울","부천시, 서초구, 종로구",www.catholic.ac.kr
...,...,...,...,...,...,...
302,호남대학교,대학교,사립,광주,광산구,www.honam.ac.kr
303,호남신학대학교,대학교,사립,광주,남구,www.htus.ac.kr
304,호산대학교,전문대학,사립,경북,경산시,www.hosan.ac.kr
305,호서대학교,대학교,사립,충남,아산시,www.hoseo.ac.kr


# 등록금을 가져온다

In [55]:
df2=pd.read_csv('한국장학재단_대학별 평균등록금_20230428.csv',encoding="EUC-KR").loc[:,'대학명':'평균등록금(원)']
col=['대학명']
if len(df2[df2['평균입학금(원)']!=0])!=0:
    col.append('평균입학금(원)')
col.append('평균등록금(원)')
df2=df2[col]
df2.rename(columns={'대학명':'학교명','평균등록금(원)':"등록금"},inplace=True)
df2

,학교명,등록금
0,강릉원주대학교,4262795
1,강원대학교,4145285
2,경북대학교,4499843
3,경상국립대학교,4042102
4,경인교육대학교,3316000
...,...,...
382,한림성심대학교,5741603
383,한양여자대학교,6189253
384,한영대학교,5611605
385,혜전대학교,5828352


# 장학금을 가져온다

In [56]:
df3=pd.read_csv('한국장학재단_대학별 장학금 수혜 현황_20230831.csv',encoding='EUC-KR')
df3=df3[['학교명','교외장학금 국가', '교외장학금 지방자치단체','교외장학금 사설및기타', '교내장학금 성적우수장학금','교내장학금 근로장학금', '교내장학금 기타장학금']]
df3['학교명']=df3['학교명'].apply(lambda x : str(x).split('(')[0])
group={}
for i in df3.columns[1:]:
    group[i]='first'
df3=df3.groupby('학교명').agg(
    group
).reset_index()
rename={}
for i in df3.columns[1:]:
    rename[i]=i.split(' ')[1]
for i in df3.columns[1:4]:
    rename[i]=rename[i]+"장학금"
df3.rename(columns=rename,inplace=True)
df3['교외장학금']=df3['국가장학금']+df3['지방자치단체장학금']+df3['사설및기타장학금']
df3['교내장학금']=df3['성적우수장학금']+df3['근로장학금']+df3['기타장학금']
df3['전체 장학금']=df3['교외장학금']+df3['교내장학금']
df3

,학교명,국가장학금,지방자치단체장학금,사설및기타장학금,성적우수장학금,근로장학금,기타장학금,교외장학금,교내장학금,전체 장학금
0,ICT폴리텍대학,280892000,0,181420000,33524000,0,0,462312000,33524000,495836000
1,가야대학교,4701192873,5438560,46300000,592548990,117497462,474536000,4752931433,1184582452,5937513885
2,가천대학교,38565866879,97550440,1361531940,6279840000,1445094011,11842199040,40024949259,19567133051,59592082310
3,가톨릭관동대학교,17562903302,430489570,664507320,2045134790,236506774,3206513498,18657900192,5488155062,24146055254
4,가톨릭꽃동네대학교,1572115751,2330640,79208000,96244000,125041136,161735634,1653654391,383020770,2036675161
...,...,...,...,...,...,...,...,...,...,...
377,호서대학교,28201647866,475156680,289731540,4422236200,1873684248,5366079773,28966536086,11662000221,40628536307
378,호원대학교,16766637439,38828280,41412000,1181016900,223946596,5158849995,16846877719,6563813491,23410691210
379,홍익대학교,32583356967,50567180,959136530,17648206300,4354288864,6442353102,33593060677,28444848266,62037908943
380,화성의과학대학교,72067036,6816400,6701000,105328000,9148564,104653360,85584436,219129924,304714360


In [57]:
money=pd.merge(df2,df3)
money['남은 장학금']=money['전체 장학금']-money['등록금']
money

,학교명,등록금,국가장학금,지방자치단체장학금,사설및기타장학금,성적우수장학금,근로장학금,기타장학금,교외장학금,교내장학금,전체 장학금,남은 장학금
0,강릉원주대학교,4262795,15808507400,181444580,464545160,2833921270,507173000,1483084310,16454497140,4824178580,21278675720,21274412925
1,강원대학교,4145285,37080248970,2787591340,1997861499,4548543150,3226232940,2483004396,41865701809,10257780486,52123482295,52119337010
2,경북대학교,4499843,41995581412,61125980,2976989350,6901994600,1753948070,2262028590,45033696742,10917971260,55951668002,55947168159
3,경상국립대학교,4042102,30664005292,198544620,958753680,3861367230,1707913842,1125682860,31821303592,6694963932,38516267524,38512225422
4,경인교육대학교,3316000,3644336917,143453400,154043950,593894000,422357548,633470560,3941834267,1649722108,5591556375,5588240375
...,...,...,...,...,...,...,...,...,...,...,...,...
369,한림성심대학교,5741603,8047162173,68170000,61276000,732140370,507962292,1199537750,8176608173,2439640412,10616248585,10610506982
370,한양여자대학교,6189253,17917173144,22246400,155133050,2032876270,275404666,1287440498,18094552594,3595721434,21690274028,21684084775
371,한영대학교,5611605,3424037092,0,0,76860000,27476398,661850750,3424037092,766187148,4190224240,4184612635
372,혜전대학교,5828352,7001190950,0,106307250,126498800,279306400,1328084425,7107498200,1733889625,8841387825,8835559473


In [80]:
result

,학교명,학제,설립구분,시,지역,학교홈페이지,등록금,국가장학금,지방자치단체장학금,사설및기타장학금,성적우수장학금,근로장학금,기타장학금,교외장학금,교내장학금,전체 장학금,남은 장학금
34,계명대학교,대학교,사립,대구,달서구,www.kmu.ac.kr,7279405,54009173885,153699890,771666681,11627885300,1855773718,9094094724,54934540456,22577753742,77512294198,77505014793
188,영남대학교,대학교,사립,경북,경산시,www.yu.ac.kr,7461513,54010997840,132613260,1535399100,10259596300,647894575,5118833280,55679010200,16026324155,71705334355,71697872842
64,단국대학교,대학교,사립,"충남, 경기","용인시, 천안시",www.dankook.ac.kr,8073411,47759877507,155146740,1236818110,2154722484,503333458,16113787423,49151842357,18771843365,67923685722,67915612311
65,단국대학교,대학교,사립,"충남, 경기","용인시, 천안시",www.dankook.ac.kr,8222972,47759877507,155146740,1236818110,2154722484,503333458,16113787423,49151842357,18771843365,67923685722,67915462750
33,경희대학교,대학교,사립,서울,동대문구,www.khu.ac.kr,7991355,43310114133,379633340,3000868177,9223085908,560974850,11262940624,46690615650,21047001382,67737617032,67729625677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,중앙승가대학교,대학교,사립,경기,김포시,admission.sangha.ac.kr,1760000,116746680,0,204052000,8850000,0,24090000,320798680,32940000,353738680,351978680
83,대전신학대학교,대학교,사립,대전,대덕구,www.daejeon.ac.kr,5956000,262337121,0,11637000,16055000,4072024,14389000,273974121,34516024,308490145,302534145
193,영산선학대학교,대학교,사립,전남,영광군,www.youngsan.ac.kr,2000000,0,0,41200000,0,0,74000000,41200000,74000000,115200000,113200000
79,대전가톨릭대학교,대학교,사립,세종,전의면,dcatholic.ac.kr,5760000,62265000,0,2000000,0,3000000,0,64265000,3000000,67265000,61505000


# 합친다

In [58]:
result=pd.merge(df1,money)

In [72]:
code=""
for i in result.columns[6:]:
    code+=f'<option value="{i}">{i}</option>\n'
clipboard.copy(code)

In [77]:
f"상관계수 : {result[['등록금','근로장학금']].corr()['등록금'].loc['근로장학금']}"

'상관계수 : 0.16211293782753228'

In [ ]:
result.to_csv('result.csv',index=False)